# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

GogleDrive-Link for the Model(.h5 files) with Good Accuracy <br>

CNN+RNN Stack using GRU -- train accuracy = 93.84% And Validation Accuracy = 93.75% <br>
https://drive.google.com/open?id=1JSUj7mxOwbD-nqseNIVZwWDEDMY_JuO2 <br>

Conv3D  -- train accuracy = 79.2% And Validation Accuracy = 81.2% <br>
https://drive.google.com/open?id=1T27pUpS-hVxGqhUQDwhmzRqRvCW3dXFs


In [1]:
#Import Libraries

In [2]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [3]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [4]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size = 32

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [5]:
n_classes = 5 #left swipe, right swipe, thumbs up, thumbs down, stop
n_channel = 3
x = 30 # number of frames
y = 120 # image width
z = 120 # image height


In [5]:

def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
     #create a list of image numbers you want to use for a particular video
    img_idx = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,n_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,n_classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    tempimg = imresize(image,(y,z))
                     
                    tempimg = tempimg/127.5-1 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (tempimg[:,:,0]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (tempimg[:,:,1]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (tempimg[:,:,2]) #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,n_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,n_classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    tempimg = imresize(image,(y,z))
                     
                    tempimg = tempimg/127.5-1 #Normalize data
                                        
                    batch_data[folder,idx,:,:,0] = (tempimg[:,:,0])
                    batch_data[folder,idx,:,:,1] = (tempimg[:,:,1])
                    batch_data[folder,idx,:,:,2] = (tempimg[:,:,2])
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [8]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 20 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 20


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [23]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout,LSTM
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
import keras

### Exp :1- Model1 with number of frames,image width,image height are (30,120,120)


In [14]:


#write your model here
#model 1
model_1 = Sequential()

model_1.add(Conv3D(8, #number of filters 
                 kernel_size=(3,3,3), 
                 input_shape=(30, 120, 120, 3),
                 padding='same'))
model_1.add(BatchNormalization())
model_1.add(Activation('relu'))

model_1.add(MaxPooling3D(pool_size=(2,2,2)))

model_1.add(Conv3D(16, #Number of filters, 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_1.add(BatchNormalization())
model_1.add(Activation('relu'))

model_1.add(MaxPooling3D(pool_size=(2,2,2)))

model_1.add(Conv3D(32, #Number of filters 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_1.add(BatchNormalization())
model_1.add(Activation('relu'))

model_1.add(MaxPooling3D(pool_size=(2,2,2)))

model_1.add(Conv3D(64, #Number pf filters 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_1.add(BatchNormalization())
model_1.add(Activation('relu'))

model_1.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_1.add(Flatten())

model_1.add(Dense(1000, activation='relu'))
model_1.add(Dropout(0.5))

model_1.add(Dense(500, activation='relu'))
model_1.add(Dropout(0.5))

#softmax layer
model_1.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [15]:
optimiser = optimizers.Adam(lr=0.001) #write your optimizer
model_1.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_1 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_2 (Batch (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_2 (Activation)    (None, 15, 60, 60, 16)    0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [16]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [9]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [17]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [18]:
model_1.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 32
Source path =  ./Project_data/train ; batch size = 32
Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 3/21 [===>..........................] - ETA: 1:31 - loss: 6.0817 - categorical_accuracy: 0.2083Batch:  4 Index: 32


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


21/21 [==============================] - 83s 4s/step - loss: 4.6679 - categorical_accuracy: 0.2801 - val_loss: 2.4683 - val_categorical_accuracy: 0.3300

Epoch 00001: saving model to model_init_2019-09-2215_29_19.738318/model-00001-4.69905-0.27753-2.46831-0.33000.h5
Epoch 2/20
21/21 [==============================] - 51s 2s/step - loss: 2.2017 - categorical_accuracy: 0.3706 - val_loss: 6.8251 - val_categorical_accuracy: 0.0625

Epoch 00002: saving model to model_init_2019-09-2215_29_19.738318/model-00002-2.20166-0.37060-6.82512-0.06250.h5
Epoch 3/20
21/21 [==============================] - 49s 2s/step - loss: 1.6493 - categorical_accuracy: 0.4177 - val_loss: 1.6799 - val_categorical_accuracy: 0.3125

Epoch 00003: saving model to model_init_2019-09-2215_29_19.738318/model-00003-1.64765-0.41920-1.67987-0.31250.h5
Epoch 4/20
21/21 [==============================] - 46s 2s/step - loss: 1.3601 - categorical_accuracy: 0.4185 - val_loss: 1.4413 - val_categorical_accuracy: 0.3125

Epoch 00004:

### Exp :2-  Model2 with same number of frames,image width,image height

In [26]:
n_classes = 5 #left swipe, right swipe, thumbs up, thumbs down, stop
n_channel = 3
x = 30 # number of frames
y = 120 # image width
z = 120 # image height


In [27]:
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

# Define model b
model_2 = Sequential()
model_2.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,n_channel), padding='same'))
model_2.add(Activation('relu'))
model_2.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model_2.add(Activation('relu'))
model_2.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_2.add(Dropout(0.25))

model_2.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_2.add(Activation('relu'))
model_2.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_2.add(Activation('relu'))
model_2.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_2.add(Dropout(0.25))

model_2.add(Flatten())
model_2.add(Dense(512, activation='relu'))
model_2.add(Dropout(0.5))
model_2.add(Dense(n_classes, activation='softmax'))

model_2.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_13 (Conv3D)           (None, 30, 120, 120, 32)  2624      
_________________________________________________________________
activation_13 (Activation)   (None, 30, 120, 120, 32)  0         
_________________________________________________________________
conv3d_14 (Conv3D)           (None, 30, 120, 120, 32)  27680     
_________________________________________________________________
activation_14 (Activation)   (None, 30, 120, 120, 32)  0         
_________________________________________________________________
max_pooling3d_9 (MaxPooling3 (None, 10, 40, 40, 32)    0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 10, 40, 40, 32)    0         
_________________________________________________________________
conv3d_15 (Conv3D)           (None, 10, 40, 40, 64)    55360     
__________

In [28]:
batch_size=10
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [29]:
num_epochs=10
model_2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 10
Source path =  ./Project_data/train ; batch size = 10
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


21/21 [==============================] - 47s 2s/step - loss: 1.9322 - categorical_accuracy: 0.1619 - val_loss: 1.5932 - val_categorical_accuracy: 0.2250

Epoch 00001: saving model to model_init_2019-09-2215_29_19.738318/model-00001-1.93221-0.16190-1.59323-0.22500.h5
Epoch 2/10
21/21 [==============================] - 38s 2s/step - loss: 1.5955 - categorical_accuracy: 0.1524 - val_loss: 1.6047 - val_categorical_accuracy: 0.2000

Epoch 00002: saving model to model_init_2019-09-2215_29_19.738318/model-00002-1.59546-0.15238-1.60470-0.20000.h5
Epoch 3/10
14/21 [===================>..........] - ETA: 11s - loss: 1.5958 - categorical_accuracy: 0.2429Batch:  67 Index: 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


21/21 [==============================] - 38s 2s/step - loss: 1.5780 - categorical_accuracy: 0.2286 - val_loss: 1.4913 - val_categorical_accuracy: 0.2500

Epoch 00003: saving model to model_init_2019-09-2215_29_19.738318/model-00003-1.57804-0.22857-1.49128-0.25000.h5
Epoch 4/10
21/21 [==============================] - 21s 999ms/step - loss: 1.5758 - categorical_accuracy: 0.2290 - val_loss: 1.5503 - val_categorical_accuracy: 0.2250

Epoch 00004: saving model to model_init_2019-09-2215_29_19.738318/model-00004-1.55908-0.23810-1.55026-0.22500.h5
Epoch 5/10
21/21 [==============================] - 15s 707ms/step - loss: 1.5128 - categorical_accuracy: 0.3333 - val_loss: 1.6004 - val_categorical_accuracy: 0.2000

Epoch 00005: saving model to model_init_2019-09-2215_29_19.738318/model-00005-1.51279-0.33333-1.60041-0.20000.h5

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 6/10
21/21 [==============================] - 15s 697ms/step - loss: 1.5277 - catego

### Exp :3- Model 2 with Changing number of frames,image width,image height values


In [10]:
n_classes = 5 #left swipe, right swipe, thumbs up, thumbs down, stop
n_channel = 3
x = 30 # number of frames
y = 60 # image width
z = 60 # image height

In [11]:
# Define model b
model_2 = Sequential()
model_2.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,n_channel), padding='same'))
model_2.add(Activation('relu'))
model_2.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model_2.add(Activation('relu'))
model_2.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_2.add(Dropout(0.25))

model_2.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_2.add(Activation('relu'))
model_2.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_2.add(Activation('relu'))
model_2.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_2.add(Dropout(0.25))

model_2.add(Flatten())
model_2.add(Dense(512, activation='relu'))
model_2.add(Dropout(0.5))
model_2.add(Dense(n_classes, activation='softmax'))

model_2.compile(optimizer=keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 60, 60, 32)    2624      
_________________________________________________________________
activation_1 (Activation)    (None, 30, 60, 60, 32)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 30, 60, 60, 32)    27680     
_________________________________________________________________
activation_2 (Activation)    (None, 30, 60, 60, 32)    0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 10, 20, 20, 32)    0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 20, 20, 32)    0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 10, 20, 20, 64)    55360     
__________

In [33]:
batch_size=10
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
num_epochs=10

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [34]:
model_2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 10
Source path =  ./Project_data/train ; batch size = 10
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


21/21 [==============================] - 25s 1s/step - loss: 1.6467 - categorical_accuracy: 0.2000 - val_loss: 1.6087 - val_categorical_accuracy: 0.2000

Epoch 00001: saving model to model_init_2019-09-2215_29_19.738318/model-00001-1.64673-0.20000-1.60869-0.20000.h5
Epoch 2/10
21/21 [==============================] - 21s 1s/step - loss: 1.6048 - categorical_accuracy: 0.2381 - val_loss: 1.5930 - val_categorical_accuracy: 0.2250

Epoch 00002: saving model to model_init_2019-09-2215_29_19.738318/model-00002-1.60476-0.23810-1.59303-0.22500.h5
Epoch 3/10
21/21 [==============================] - 22s 1s/step - loss: 1.6097 - categorical_accuracy: 0.1857 - val_loss: 1.6128 - val_categorical_accuracy: 0.1250

Epoch 00003: saving model to model_init_2019-09-2215_29_19.738318/model-00003-1.60972-0.18571-1.61284-0.12500.h5
Epoch 4/10
 2/21 [=>............................] - ETA: 24s - loss: 1.6124 - categorical_accuracy: 0.1000Batch:  67 Index: 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


21/21 [==============================] - 11s 510ms/step - loss: 1.6053 - categorical_accuracy: 0.2655 - val_loss: 1.6116 - val_categorical_accuracy: 0.1250

Epoch 00004: saving model to model_init_2019-09-2215_29_19.738318/model-00004-1.60547-0.27381-1.61161-0.12500.h5

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 5/10
21/21 [==============================] - 7s 349ms/step - loss: 1.5392 - categorical_accuracy: 0.3968 - val_loss: 1.7682 - val_categorical_accuracy: 0.3250

Epoch 00005: saving model to model_init_2019-09-2215_29_19.738318/model-00005-1.53917-0.39683-1.76821-0.32500.h5
Epoch 6/10
21/21 [==============================] - 7s 337ms/step - loss: 1.6486 - categorical_accuracy: 0.1587 - val_loss: 1.6026 - val_categorical_accuracy: 0.2500

Epoch 00006: saving model to model_init_2019-09-2215_29_19.738318/model-00006-1.64863-0.15873-1.60260-0.25000.h5

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
Epoch 7/

### Exp :4- Model 2 with same number of frames,image width,image height values but changing Batch size  to 20 


In [36]:
train_generator = generator(train_path, train_doc, 20)
val_generator = generator(val_path, val_doc, 20)
num_epochs=20

In [37]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [38]:
model_2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 20
Source path =  ./Project_data/train ; batch size = 20
Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


32/67 [=============>................] - ETA: 1:17 - loss: 1.6083 - categorical_accuracy: 0.2078Batch:  34 Index: 20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 84s 1s/step - loss: 1.6055 - categorical_accuracy: 0.2207 - val_loss: 1.6086 - val_categorical_accuracy: 0.2100

Epoch 00001: saving model to model_init_2019-09-2215_29_19.738318/model-00001-1.60748-0.20997-1.60862-0.21000.h5
Epoch 2/20
67/67 [==============================] - 28s 416ms/step - loss: 1.6114 - categorical_accuracy: 0.1891 - val_loss: 1.6083 - val_categorical_accuracy: 0.2100

Epoch 00002: saving model to model_init_2019-09-2215_29_19.738318/model-00002-1.61142-0.18905-1.60830-0.21000.h5
Epoch 3/20
67/67 [==============================] - 29s 438ms/step - loss: 1.6084 - categorical_accuracy: 0.1841 - val_loss: 1.6068 - val_categorical_accuracy: 0.2100

Epoch 00003: saving model to model_init_2019-09-2215_29_19.738318/model-00003-1.60840-0.18408-1.60683-0.21000.h5
Epoch 4/20
67/67 [==============================] - 29s 440ms/step - loss: 1.6031 - categorical_accuracy: 0.2736 - val_loss: 1.6006 - val_categorical_accuracy: 0.2600

Epo

### Exp :5- Model 2 with same number of frames,image width,image height values Changing Batch size  to 30 

In [12]:
train_generator = generator(train_path, train_doc, 30)
val_generator = generator(val_path, val_doc, 30)
num_epochs=10

In [13]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [14]:
model_2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 30
Source path =  ./Project_data/train ; batch size = 30
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 1/21 [>.............................] - ETA: 4:46 - loss: 1.6094 - categorical_accuracy: 0.2333Batch:  4 Index: 30


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


21/21 [==============================] - 73s 3s/step - loss: 1.7201 - categorical_accuracy: 0.2238 - val_loss: 1.6021 - val_categorical_accuracy: 0.3200

Epoch 00001: saving model to model_init_2019-09-2216_44_36.385184/model-00001-1.72011-0.22381-1.60207-0.32000.h5
Epoch 2/10
Batch:  23 Index: 30
21/21 [==============================] - 9s 437ms/step - loss: 1.6036 - categorical_accuracy: 0.2530 - val_loss: 1.5997 - val_categorical_accuracy: 0.1750

Epoch 00002: saving model to model_init_2019-09-2216_44_36.385184/model-00002-1.60301-0.26667-1.59966-0.17500.h5
Epoch 3/10
21/21 [==============================] - 8s 378ms/step - loss: 1.6344 - categorical_accuracy: 0.1111 - val_loss: 1.6084 - val_categorical_accuracy: 0.2500

Epoch 00003: saving model to model_init_2019-09-2216_44_36.385184/model-00003-1.63442-0.11111-1.60843-0.25000.h5
Epoch 4/10
21/21 [==============================] - 7s 341ms/step - loss: 1.6097 - categorical_accuracy: 0.2063 - val_loss: 1.6113 - val_categorical_acc

### Exp :6- Model 2 with same number of frames,image width,image height values Changing Batch size  to 40 

In [15]:
train_generator = generator(train_path, train_doc, 40)
val_generator = generator(val_path, val_doc, 40)
num_epochs=10

In [16]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [17]:
model_2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 40
Source path =  ./Project_data/train ; batch size = 40
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 1/21 [>.............................] - ETA: 3:31 - loss: 1.6081 - categorical_accuracy: 0.1750Batch:  3 Index: 40


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


21/21 [==============================] - 86s 4s/step - loss: 1.6091 - categorical_accuracy: 0.2072 - val_loss: 1.6077 - val_categorical_accuracy: 0.2417

Epoch 00001: saving model to model_init_2019-09-2216_44_36.385184/model-00001-1.60899-0.20795-1.60772-0.24167.h5
Epoch 2/10
21/21 [==============================] - 51s 2s/step - loss: 1.6090 - categorical_accuracy: 0.2050 - val_loss: 1.6094 - val_categorical_accuracy: 0.1875

Epoch 00002: saving model to model_init_2019-09-2216_44_36.385184/model-00002-1.60896-0.20497-1.60944-0.18750.h5
Epoch 3/10
21/21 [==============================] - 43s 2s/step - loss: 1.6087 - categorical_accuracy: 0.2082 - val_loss: 1.6093 - val_categorical_accuracy: 0.2125

Epoch 00003: saving model to model_init_2019-09-2216_44_36.385184/model-00003-1.60874-0.20681-1.60931-0.21250.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.
Epoch 4/10
21/21 [==============================] - 40s 2s/step - loss: 1.6097 - categorical_acc

### Exp :7- Model 2 with same number of frames,image width,image height values but Change Optimizer to Adadelta

In [18]:
model_2.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 60, 60, 32)    2624      
_________________________________________________________________
activation_1 (Activation)    (None, 30, 60, 60, 32)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 30, 60, 60, 32)    27680     
_________________________________________________________________
activation_2 (Activation)    (None, 30, 60, 60, 32)    0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 10, 20, 20, 32)    0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 20, 20, 32)    0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 10, 20, 20, 64)    55360     
__________

In [19]:
train_generator = generator(train_path, train_doc, 40)
val_generator = generator(val_path, val_doc, 40)

In [20]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [21]:
model_2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path = Source path =   ./Project_data/val./Project_data/train ; batch size = 40
Epoch 1/10
 ; batch size = 40


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 1/21 [>.............................] - ETA: 2:39 - loss: 1.6069 - categorical_accuracy: 0.2000Batch:  3 Index: 40


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


21/21 [==============================] - 83s 4s/step - loss: 1.6101 - categorical_accuracy: 0.1826 - val_loss: 1.6089 - val_categorical_accuracy: 0.2417

Epoch 00001: saving model to model_init_2019-09-2216_44_36.385184/model-00001-1.60988-0.18543-1.60890-0.24167.h5
Epoch 2/10
21/21 [==============================] - 49s 2s/step - loss: 1.6074 - categorical_accuracy: 0.2277 - val_loss: 1.6092 - val_categorical_accuracy: 0.2000

Epoch 00002: saving model to model_init_2019-09-2216_44_36.385184/model-00002-1.60736-0.22774-1.60916-0.20000.h5
Epoch 3/10
21/21 [==============================] - 43s 2s/step - loss: 1.6120 - categorical_accuracy: 0.1957 - val_loss: 1.6079 - val_categorical_accuracy: 0.2250

Epoch 00003: saving model to model_init_2019-09-2216_44_36.385184/model-00003-1.61231-0.19221-1.60789-0.22500.h5
Epoch 4/10
21/21 [==============================] - 42s 2s/step - loss: 1.6122 - categorical_accuracy: 0.1868 - val_loss: 1.6055 - val_categorical_accuracy: 0.2625

Epoch 00004:

### Exp :8- Model 3 with Change back number of frames,image width,image height values to (30,120,120)

In [35]:
# Changing back batch size, images per frame, height and width of image
batch_size = 10
x = 30 # number of frames
y = 120 # image width
z = 120# image height
num_epochs = 20

In [36]:
model_3 = Sequential()
model_3.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,n_channel), padding="same"))
model_3.add(Activation('relu'))
model_3.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
model_3.add(Activation('relu'))
model_3.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_3.add(Dropout(0.25))

model_3.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_3.add(Activation('relu'))
model_3.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_3.add(Activation('relu'))
model_3.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_3.add(Dropout(0.25))

model_3.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_3.add(Activation('relu'))
model_3.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_3.add(Activation('relu'))
model_3.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_3.add(Dropout(0.25))

model_3.add(Flatten())
model_3.add(Dense(512, activation='relu'))
model_3.add(BatchNormalization())
model_3.add(Dropout(0.5))
model_3.add(Dense(n_classes, activation='softmax'))
model_3.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_13 (Conv3D)           (None, 30, 120, 120, 32)  2624      
_________________________________________________________________
activation_13 (Activation)   (None, 30, 120, 120, 32)  0         
_________________________________________________________________
conv3d_14 (Conv3D)           (None, 30, 120, 120, 32)  27680     
_________________________________________________________________
activation_14 (Activation)   (None, 30, 120, 120, 32)  0         
_________________________________________________________________
max_pooling3d_9 (MaxPooling3 (None, 10, 40, 40, 32)    0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 10, 40, 40, 32)    0         
_________________________________________________________________
conv3d_15 (Conv3D)           (None, 10, 40, 40, 64)    55360     
__________

In [37]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [38]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [39]:
model_3.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 10
Source path =  ./Project_data/train ; batch size = 10
Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


56/67 [========================>.....] - ETA: 20s - loss: 2.1865 - categorical_accuracy: 0.3071Batch:  67 Index: 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 127s 2s/step - loss: 2.1501 - categorical_accuracy: 0.3039 - val_loss: 4.3592 - val_categorical_accuracy: 0.3200

Epoch 00001: saving model to model_init_2019-09-2115_12_34.066620/model-00001-2.16010-0.30015-4.35920-0.32000.h5
Epoch 2/20
67/67 [==============================] - 47s 706ms/step - loss: 2.1513 - categorical_accuracy: 0.2388 - val_loss: 2.3216 - val_categorical_accuracy: 0.2400

Epoch 00002: saving model to model_init_2019-09-2115_12_34.066620/model-00002-2.15131-0.23881-2.32159-0.24000.h5
Epoch 3/20
67/67 [==============================] - 47s 703ms/step - loss: 1.9491 - categorical_accuracy: 0.2786 - val_loss: 2.3894 - val_categorical_accuracy: 0.2700

Epoch 00003: saving model to model_init_2019-09-2115_12_34.066620/model-00003-1.94913-0.27861-2.38944-0.27000.h5
Epoch 4/20
67/67 [==============================] - 47s 704ms/step - loss: 1.8350 - categorical_accuracy: 0.2886 - val_loss: 1.4254 - val_categorical_accuracy: 0.4400

Ep

### Exp :9- Model 4 with same number of frames,image width,image height values 

In [40]:
input_shape=(x,y,z,n_channel)

nb_filters = [8,16,32,64]
nb_dense = [1000, 500, 5]
# Define model
model_4 = Sequential()

model_4.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_4.add(BatchNormalization())
model_4.add(Activation('relu'))

model_4.add(MaxPooling3D(pool_size=(2,2,2)))

model_4.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_4.add(BatchNormalization())
model_4.add(Activation('relu'))

model_4.add(MaxPooling3D(pool_size=(2,2,2)))

model_4.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_4.add(BatchNormalization())
model_4.add(Activation('relu'))

model_4.add(MaxPooling3D(pool_size=(2,2,2)))

model_4.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_4.add(BatchNormalization())
model_4.add(Activation('relu'))

model_4.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_4.add(Flatten())

model_4.add(Dense(nb_dense[0], activation='relu'))
model_4.add(Dropout(0.5))

model_4.add(Dense(nb_dense[1], activation='relu'))
model_4.add(Dropout(0.5))

#softmax layer
model_4.add(Dense(nb_dense[2], activation='softmax'))
model_4.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_4.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_19 (Conv3D)           (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_6 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_19 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_12 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_20 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_7 (Batch (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_20 (Activation)   (None, 15, 60, 60, 16)    0         
__________

In [41]:
batch_size = 10
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
num_epochs = 10

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model_4.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 10
Source path =  ./Project_data/train ; batch size =Epoch 1/10
 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


65/67 [============================>.] - ETA: 2s - loss: 5.7044 - categorical_accuracy: 0.2523Batch:  67 Index: 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 79s 1s/step - loss: 5.6102 - categorical_accuracy: 0.2493 - val_loss: 2.6864 - val_categorical_accuracy: 0.2800

Epoch 00001: saving model to model_init_2019-09-2115_12_34.066620/model-00001-5.64189-0.25189-2.68638-0.28000.h5
Epoch 2/10
67/67 [==============================] - 24s 359ms/step - loss: 2.5352 - categorical_accuracy: 0.3184 - val_loss: 1.6390 - val_categorical_accuracy: 0.3600

Epoch 00002: saving model to model_init_2019-09-2115_12_34.066620/model-00002-2.53516-0.31841-1.63904-0.36000.h5
Epoch 3/10
67/67 [==============================] - 27s 401ms/step - loss: 1.9549 - categorical_accuracy: 0.2637 - val_loss: 1.3116 - val_categorical_accuracy: 0.4400

Epoch 00003: saving model to model_init_2019-09-2115_12_34.066620/model-00003-1.95494-0.26368-1.31159-0.44000.h5
Epoch 4/10
67/67 [==============================] - 25s 373ms/step - loss: 1.7522 - categorical_accuracy: 0.2985 - val_loss: 1.2079 - val_categorical_accuracy: 0.5000

Epo

### Exp :10- Model 5 with Changing number of frames,image width,image height values 

In [42]:
x = 30 # number of frames
y = 64 # image width
z = 64 # image height 

n_classes = 5
n_channel = 3

In [45]:
model_5 = Sequential()
model_5.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,n_channel), padding="same"))
model_5.add(Activation('relu'))
model_5.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
model_5.add(Activation('relu'))
model_5.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_5.add(Dropout(0.25))

model_5.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_5.add(Activation('relu'))
model_5.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_5.add(Activation('relu'))
model_5.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_5.add(Dropout(0.25))

model_5.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_5.add(Activation('relu'))
model_5.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_5.add(Activation('relu'))
model_5.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_5.add(Dropout(0.25))

model_5.add(Flatten())
model_5.add(Dense(512, activation='relu'))
model_5.add(BatchNormalization())
model_5.add(Dropout(0.5))
model_5.add(Dense(n_classes, activation='softmax'))
model_5.compile(optimizer=optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_5.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_23 (Conv3D)           (None, 30, 64, 64, 32)    2624      
_________________________________________________________________
activation_23 (Activation)   (None, 30, 64, 64, 32)    0         
_________________________________________________________________
conv3d_24 (Conv3D)           (None, 30, 64, 64, 32)    27680     
_________________________________________________________________
activation_24 (Activation)   (None, 30, 64, 64, 32)    0         
_________________________________________________________________
max_pooling3d_16 (MaxPooling (None, 10, 22, 22, 32)    0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 10, 22, 22, 32)    0         
_________________________________________________________________
conv3d_25 (Conv3D)           (None, 10, 22, 22, 64)    55360     
__________

In [46]:
batch_size = 20
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
num_epochs = 20

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model_5.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 20
Source path =  ./Project_data/train ; batch size = 20
Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


32/34 [===========================>..] - ETA: 4s - loss: 2.1559 - categorical_accuracy: 0.2844Batch:  34 Index: 20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


34/34 [==============================] - 75s 2s/step - loss: 2.1077 - categorical_accuracy: 0.3017 - val_loss: 1.6028 - val_categorical_accuracy: 0.3700

Epoch 00001: saving model to model_init_2019-09-2115_12_34.066620/model-00001-2.14180-0.29261-1.60282-0.37000.h5
Epoch 2/20
34/34 [==============================] - 10s 297ms/step - loss: 2.2930 - categorical_accuracy: 0.1961 - val_loss: 7.3864 - val_categorical_accuracy: 0.2300

Epoch 00002: saving model to model_init_2019-09-2115_12_34.066620/model-00002-2.29299-0.19608-7.38641-0.23000.h5
Epoch 3/20
34/34 [==============================] - 13s 383ms/step - loss: 2.2068 - categorical_accuracy: 0.2451 - val_loss: 6.7557 - val_categorical_accuracy: 0.2300

Epoch 00003: saving model to model_init_2019-09-2115_12_34.066620/model-00003-2.20682-0.24510-6.75566-0.23000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.5.
Epoch 4/20
34/34 [==============================] - 13s 373ms/step - loss: 1.9834 - categorical_accuracy: 0.

### Exp :11- Model 6 

In [47]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

n_classes = 5
n_channel = 3
nb_filters = [8,16,32,64]
nb_dense = [1000, 500, 5]

input_shape=(x,y,z,n_channel)

# Define model
model_6 = Sequential()

model_6.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_6.add(BatchNormalization())
model_6.add(Activation('relu'))

model_6.add(MaxPooling3D(pool_size=(2,2,2)))

model_6.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_6.add(BatchNormalization())
model_6.add(Activation('relu'))

model_6.add(MaxPooling3D(pool_size=(2,2,2)))

model_6.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_6.add(BatchNormalization())
model_6.add(Activation('relu'))

model_6.add(MaxPooling3D(pool_size=(2,2,2)))

model_6.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_6.add(BatchNormalization())
model_6.add(Activation('relu'))

model_6.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_6.add(Flatten())

model_6.add(Dense(nb_dense[0], activation='relu'))
model_6.add(Dropout(0.5))

model_6.add(Dense(nb_dense[1], activation='relu'))
model_6.add(Dropout(0.5))

#softmax layer
model_6.add(Dense(nb_dense[2], activation='softmax'))
model_6.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_6.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_29 (Conv3D)           (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_11 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_29 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_19 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_30 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_12 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_30 (Activation)   (None, 15, 60, 60, 16)    0         
__________

In [48]:
batch_size = 10
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
num_epochs = 30
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

model_6.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 10
Source path =  ./Project_data/train ; batch size = 10
Epoch 1/30


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


65/67 [============================>.] - ETA: 2s - loss: 6.4483 - categorical_accuracy: 0.2646Batch:  67 Index: 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 77s 1s/step - loss: 6.3179 - categorical_accuracy: 0.2677 - val_loss: 3.0815 - val_categorical_accuracy: 0.3100

Epoch 00001: saving model to model_init_2019-09-2115_12_34.066620/model-00001-6.36483-0.26697-3.08149-0.31000.h5
Epoch 2/30
67/67 [==============================] - 22s 334ms/step - loss: 2.4801 - categorical_accuracy: 0.1940 - val_loss: 1.8449 - val_categorical_accuracy: 0.3200

Epoch 00002: saving model to model_init_2019-09-2115_12_34.066620/model-00002-2.48015-0.19403-1.84487-0.32000.h5
Epoch 3/30
67/67 [==============================] - 25s 379ms/step - loss: 1.8412 - categorical_accuracy: 0.2985 - val_loss: 1.7328 - val_categorical_accuracy: 0.2400

Epoch 00003: saving model to model_init_2019-09-2115_12_34.066620/model-00003-1.84117-0.29851-1.73282-0.24000.h5
Epoch 4/30
67/67 [==============================] - 25s 367ms/step - loss: 1.6102 - categorical_accuracy: 0.3682 - val_loss: 1.6257 - val_categorical_accuracy: 0.2500

Epo

67/67 [==============================] - 24s 354ms/step - loss: 0.7085 - categorical_accuracy: 0.7015 - val_loss: 0.6650 - val_categorical_accuracy: 0.7600

Epoch 00029: saving model to model_init_2019-09-2115_12_34.066620/model-00029-0.70849-0.70149-0.66503-0.76000.h5
Epoch 30/30
67/67 [==============================] - 25s 374ms/step - loss: 0.6580 - categorical_accuracy: 0.7214 - val_loss: 0.6604 - val_categorical_accuracy: 0.7700

Epoch 00030: saving model to model_init_2019-09-2115_12_34.066620/model-00030-0.65798-0.72139-0.66042-0.77000.h5


### Final Model - Conv3D

In [11]:
#n_filters = [8,16,32,64]
#n_dense = [256, 128, 5]

x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

n_classes = 5
n_channel = 3

# Input
input_shape=(x,y,z,n_channel)

# Define model
model_f = Sequential()

model_f.add(Conv3D(8, 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_f.add(BatchNormalization())
model_f.add(Activation('relu'))

model_f.add(MaxPooling3D(pool_size=(2,2,2)))

model_f.add(Conv3D(16, 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_f.add(BatchNormalization())
model_f.add(Activation('relu'))

model_f.add(MaxPooling3D(pool_size=(2,2,2)))

model_f.add(Conv3D(32, 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_f.add(BatchNormalization())
model_f.add(Activation('relu'))

model_f.add(MaxPooling3D(pool_size=(2,2,2)))

model_f.add(Conv3D(64, 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_f.add(Activation('relu'))
model_f.add(Dropout(0.25))

model_f.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_f.add(Flatten())

model_f.add(Dense(256, activation='relu'))
model_f.add(Dropout(0.5))

model_f.add(Dense(128, activation='relu'))
model_f.add(Dropout(0.5))

#softmax layer
model_f.add(Dense(5, activation='softmax'))

In [12]:
optimiser = optimizers.Adam() #write your optimizer
model_f.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_f.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_5 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_4 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_5 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_5 (Batch (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_6 (Activation)    (None, 15, 60, 60, 16)    0         
__________

In [24]:
batch_size = 10
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [25]:

num_epochs = 20

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
model_f.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 10
Source path =  ./Project_data/train ; batch size = 10
Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


65/67 [============================>.] - ETA: 2s - loss: 1.9930 - categorical_accuracy: 0.2523Batch:  67 Index: 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 84s 1s/step - loss: 1.9784 - categorical_accuracy: 0.2587 - val_loss: 1.4354 - val_categorical_accuracy: 0.2900

Epoch 00001: saving model to model_init_2019-09-2216_44_36.385184/model-00001-1.98052-0.25792-1.43541-0.29000.h5
Epoch 2/20
67/67 [==============================] - 24s 355ms/step - loss: 1.6124 - categorical_accuracy: 0.2338 - val_loss: 1.5106 - val_categorical_accuracy: 0.2000

Epoch 00002: saving model to model_init_2019-09-2216_44_36.385184/model-00002-1.61238-0.23383-1.51060-0.20000.h5
Epoch 3/20
67/67 [==============================] - 26s 394ms/step - loss: 1.5303 - categorical_accuracy: 0.2935 - val_loss: 1.4576 - val_categorical_accuracy: 0.2600

Epoch 00003: saving model to model_init_2019-09-2216_44_36.385184/model-00003-1.53030-0.29353-1.45758-0.26000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 4/20
67/67 [==============================] - 27s 401ms/step - loss: 1.5352 - catego

In [26]:
batch_size = 32
num_epochs = 20

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model_f.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 32
Source path =  Epoch 1/20
./Project_data/train ; batch size = 32


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 3/21 [===>..........................] - ETA: 1:23 - loss: 0.5692 - categorical_accuracy: 0.8438Batch:  4 Index: 32


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


21/21 [==============================] - 84s 4s/step - loss: 0.6513 - categorical_accuracy: 0.7415 - val_loss: 0.8022 - val_categorical_accuracy: 0.7300

Epoch 00001: saving model to model_init_2019-09-2216_44_36.385184/model-00001-0.65006-0.74208-0.80216-0.73000.h5
Epoch 2/20
21/21 [==============================] - 54s 3s/step - loss: 0.6524 - categorical_accuracy: 0.7433 - val_loss: 0.6856 - val_categorical_accuracy: 0.7500

Epoch 00002: saving model to model_init_2019-09-2216_44_36.385184/model-00002-0.65245-0.74327-0.68560-0.75000.h5
Epoch 3/20
21/21 [==============================] - 50s 2s/step - loss: 0.6047 - categorical_accuracy: 0.7697 - val_loss: 0.8409 - val_categorical_accuracy: 0.6250

Epoch 00003: saving model to model_init_2019-09-2216_44_36.385184/model-00003-0.60392-0.77049-0.84092-0.62500.h5
Epoch 4/20
21/21 [==============================] - 46s 2s/step - loss: 0.6052 - categorical_accuracy: 0.7870 - val_loss: 0.6090 - val_categorical_accuracy: 0.8125

Epoch 00004:

#### The Above Model Gives Desired Accuracy of 79.2% in Training and 81.2% in validation.It shows that model is stable

In [27]:
batch_size = 64
num_epochs = 20

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model_f.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/valSource path =  ./Project_data/train ; batch size = 64
Epoch 1/20
 ; batch size = 64


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


Batch:  2 Index: 64


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


11/11 [==============================] - 88s 8s/step - loss: 0.5546 - categorical_accuracy: 0.7862 - val_loss: 0.6799 - val_categorical_accuracy: 0.7900

Epoch 00001: saving model to model_init_2019-09-2216_44_36.385184/model-00001-0.54369-0.78884-0.67994-0.79000.h5
Epoch 2/20
11/11 [==============================] - 29s 3s/step - loss: 0.5766 - categorical_accuracy: 0.7866 - val_loss: 0.7588 - val_categorical_accuracy: 0.7778

Epoch 00002: saving model to model_init_2019-09-2216_44_36.385184/model-00002-0.57663-0.78656-0.75881-0.77778.h5
Epoch 3/20
11/11 [==============================] - 31s 3s/step - loss: 0.5635 - categorical_accuracy: 0.8103 - val_loss: 0.8376 - val_categorical_accuracy: 0.6607

Epoch 00003: saving model to model_init_2019-09-2216_44_36.385184/model-00003-0.56345-0.81028-0.83758-0.66071.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 9.765625463842298e-07.
Epoch 4/20
Batch:  7 Index: 16
11/11 [==============================] - 28s 3s/step - loss: 0.57

In [28]:
batch_size = 40
num_epochs = 20

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

model_f.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 40
Source path =  ./Project_data/train ; batch size = 40
Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 2/17 [==>...........................] - ETA: 1:33 - loss: 0.5346 - categorical_accuracy: 0.8125Batch:  3 Index: 40


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


17/17 [==============================] - 86s 5s/step - loss: 0.5848 - categorical_accuracy: 0.7558 - val_loss: 0.7091 - val_categorical_accuracy: 0.7700

Epoch 00001: saving model to model_init_2019-09-2216_44_36.385184/model-00001-0.58740-0.75415-0.70910-0.77000.h5
Epoch 2/20
17/17 [==============================] - 46s 3s/step - loss: 0.5077 - categorical_accuracy: 0.7928 - val_loss: 0.8271 - val_categorical_accuracy: 0.7167

Epoch 00002: saving model to model_init_2019-09-2216_44_36.385184/model-00002-0.50770-0.79284-0.82710-0.71667.h5
Epoch 3/20
17/17 [==============================] - 44s 3s/step - loss: 0.6154 - categorical_accuracy: 0.7783 - val_loss: 0.6509 - val_categorical_accuracy: 0.8000

Epoch 00003: saving model to model_init_2019-09-2216_44_36.385184/model-00003-0.61958-0.77657-0.65085-0.80000.h5
Epoch 4/20
17/17 [==============================] - 38s 2s/step - loss: 0.6418 - categorical_accuracy: 0.7337 - val_loss: 0.7181 - val_categorical_accuracy: 0.7667

Epoch 00004:

### CNN+RNN Stack using GRU

In [41]:
from keras.applications.vgg16 import VGG16
    
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

n_classes = 5
n_channel = 3

# Input
input_shape=(x,y,z,n_channel)

    
    
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(120,120,3))
cnn_out = base_model.output
cnn_out = Flatten()(cnn_out)

features = Dense(64, activation='relu')(cnn_out)
conv_model = Model(inputs=base_model.input, outputs=features)

for layer in base_model.layers:
    layer.trainable = False
        
model_gpu = Sequential()
model_gpu.add(TimeDistributed(conv_model, input_shape=input_shape))
model_gpu.add(GRU(32, return_sequences=True))
model_gpu.add(GRU(16))
model_gpu.add(Dropout(0.5))
model_gpu.add(Dense(8, activation='relu'))
model_gpu.add(Dense(5, activation='softmax'))

In [42]:
optimiser = optimizers.Adam() #write your optimizer
model_gpu.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_gpu.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_2 (TimeDist (None, 30, 64)            15009664  
_________________________________________________________________
gru_3 (GRU)                  (None, 30, 32)            9312      
_________________________________________________________________
gru_4 (GRU)                  (None, 16)                2352      
_________________________________________________________________
dropout_8 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_11 (Dense)             (None, 5)                 45        
Total params: 15,021,509
Trainable params: 306,821
Non-trainable params: 14,714,688
__________________________________________________________

In [48]:
batch_size = 10
num_epochs = 10

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

model_gpu.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 10
Source path =  ./Project_data/train ; batch size = 10
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


65/67 [============================>.] - ETA: 2s - loss: 0.5841 - categorical_accuracy: 0.8646Batch:  67 Index: 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 97s 1s/step - loss: 0.5783 - categorical_accuracy: 0.8671 - val_loss: 0.7930 - val_categorical_accuracy: 0.7700

Epoch 00001: saving model to model_init_2019-09-2217_59_43.925201/model-00001-0.58035-0.86576-0.79300-0.77000.h5
Epoch 2/10
67/67 [==============================] - 34s 506ms/step - loss: 0.5008 - categorical_accuracy: 0.9204 - val_loss: 0.8250 - val_categorical_accuracy: 0.7700

Epoch 00002: saving model to model_init_2019-09-2217_59_43.925201/model-00002-0.50078-0.92040-0.82500-0.77000.h5
Epoch 3/10
67/67 [==============================] - 34s 503ms/step - loss: 0.5476 - categorical_accuracy: 0.8706 - val_loss: 0.7480 - val_categorical_accuracy: 0.7500

Epoch 00003: saving model to model_init_2019-09-2217_59_43.925201/model-00003-0.54761-0.87065-0.74798-0.75000.h5
Epoch 4/10
67/67 [==============================] - 34s 504ms/step - loss: 0.5631 - categorical_accuracy: 0.8856 - val_loss: 0.7798 - val_categorical_accuracy: 0.7500

Epo

In [44]:
batch_size = 32
num_epochs = 20

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

model_gpu.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 32
Source path =  ./Project_data/trainEpoch 1/20
 ; batch size = 32


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


Batch:  4 Index: 32


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


21/21 [==============================] - 113s 5s/step - loss: 1.6177 - categorical_accuracy: 0.2395 - val_loss: 1.5341 - val_categorical_accuracy: 0.3500

Epoch 00001: saving model to model_init_2019-09-2217_59_43.925201/model-00001-1.61816-0.23982-1.53408-0.35000.h5
Epoch 2/20
21/21 [==============================] - 42s 2s/step - loss: 1.5071 - categorical_accuracy: 0.3478 - val_loss: 1.5092 - val_categorical_accuracy: 0.5000

Epoch 00002: saving model to model_init_2019-09-2217_59_43.925201/model-00002-1.50714-0.34783-1.50925-0.50000.h5
Epoch 3/20
21/21 [==============================] - 48s 2s/step - loss: 1.4286 - categorical_accuracy: 0.3961 - val_loss: 1.3046 - val_categorical_accuracy: 0.4375

Epoch 00003: saving model to model_init_2019-09-2217_59_43.925201/model-00003-1.43365-0.39344-1.30457-0.43750.h5
Epoch 4/20
21/21 [==============================] - 46s 2s/step - loss: 1.3240 - categorical_accuracy: 0.4637 - val_loss: 1.3253 - val_categorical_accuracy: 0.6250

Epoch 00004

####  Final Model -CNN+RNN Stack using GRU with BatchNormalization

In [28]:
from keras.applications.vgg16 import VGG16
    
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

n_classes = 5
n_channel = 3

# Input
input_shape=(x,y,z,n_channel)
   
    
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(120,120,3))
cnn_out = base_model.output
cnn_out = Flatten()(cnn_out)

features = Dense(64, activation='relu')(cnn_out)
conv_model = Model(inputs=base_model.input, outputs=features)

for layer in base_model.layers:
    layer.trainable = False
        
model_gpu1 = Sequential()
model_gpu1.add(TimeDistributed(conv_model, input_shape=input_shape))
model_gpu1.add(GRU(32, return_sequences=True))
model_gpu1.add(BatchNormalization())
model_gpu1.add(GRU(16))
model_gpu1.add(Dropout(0.5))
model_gpu1.add(Dense(8, activation='relu'))
model_gpu1.add(Dense(5, activation='softmax'))
optimiser = optimizers.Adam() #write your optimizer
model_gpu1.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_gpu1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_8 (TimeDist (None, 30, 64)            15009664  
_________________________________________________________________
gru_8 (GRU)                  (None, 30, 32)            9312      
_________________________________________________________________
batch_normalization_6 (Batch (None, 30, 32)            128       
_________________________________________________________________
gru_9 (GRU)                  (None, 16)                2352      
_________________________________________________________________
dropout_6 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_19 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_20 (Dense)             (None, 5)                 45        
Total para

In [16]:
batch_size = 32
num_epochs = 20

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

model_gpu1.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 32
Source path =  ./Project_data/train ; batch size = 32
Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


Batch:  4 Index: 32


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


21/21 [==============================] - 193s 9s/step - loss: 1.5840 - categorical_accuracy: 0.3192 - val_loss: 1.5133 - val_categorical_accuracy: 0.4100

Epoch 00001: saving model to model_init_2019-09-2309_36_12.102624/model-00001-1.58564-0.31825-1.51330-0.41000.h5
Epoch 2/20
21/21 [==============================] - 42s 2s/step - loss: 1.2766 - categorical_accuracy: 0.4990 - val_loss: 1.2000 - val_categorical_accuracy: 0.5625

Epoch 00002: saving model to model_init_2019-09-2309_36_12.102624/model-00002-1.27655-0.49896-1.19996-0.56250.h5
Epoch 3/20
21/21 [==============================] - 49s 2s/step - loss: 1.1375 - categorical_accuracy: 0.5931 - val_loss: 1.2358 - val_categorical_accuracy: 0.5000

Epoch 00003: saving model to model_init_2019-09-2309_36_12.102624/model-00003-1.14134-0.59016-1.23582-0.50000.h5
Epoch 4/20
21/21 [==============================] - 46s 2s/step - loss: 0.9753 - categorical_accuracy: 0.6867 - val_loss: 1.3010 - val_categorical_accuracy: 0.5000

Epoch 00004

#### The Above Model Gives Desired Accuracy of 93.84% in Training and 93.75% in validation,It shows model is stable

In [29]:
batch_size = 16
num_epochs = 30

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

model_gpu1.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 16
Source path =  ./Project_data/train ; batch size = 16
Epoch 1/30


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 6/42 [===>..........................] - ETA: 1:41 - loss: 1.8515 - categorical_accuracy: 0.2188

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


42/42 [==============================] - 92s 2s/step - loss: 1.5914 - categorical_accuracy: 0.3271 - val_loss: 1.4233 - val_categorical_accuracy: 0.3600

Epoch 00001: saving model to model_init_2019-09-2309_36_12.102624/model-00001-1.59453-0.32579-1.42328-0.36000.h5
Epoch 2/30
42/42 [==============================] - 32s 757ms/step - loss: 1.3335 - categorical_accuracy: 0.4558 - val_loss: 1.2119 - val_categorical_accuracy: 0.5000

Epoch 00002: saving model to model_init_2019-09-2309_36_12.102624/model-00002-1.33347-0.45578-1.21195-0.50000.h5
Epoch 3/30
42/42 [==============================] - 37s 884ms/step - loss: 1.2277 - categorical_accuracy: 0.4762 - val_loss: 1.1184 - val_categorical_accuracy: 0.4286

Epoch 00003: saving model to model_init_2019-09-2309_36_12.102624/model-00003-1.22767-0.47619-1.11845-0.42857.h5
Epoch 4/30
42/42 [==============================] - 29s 683ms/step - loss: 1.2129 - categorical_accuracy: 0.5468 - val_loss: 1.4730 - val_categorical_accuracy: 0.3929

Epo

### Insights:-

#### Final Model -CNN+RNN Stack using GRU

<font color='blue'> Batch_size = 32 and num_epochs = 20 ,
 </font><br>
<font color='red'>In 20th epoch-> train accuracy = 93.84% And Validation Accuracy = 93.75%
</font><br>

__.h5 file__<br>
In 20th epoch ==> model_init_2019-09-2309_36_12.102624/model-00020-0.34248-0.93838-0.44590-0.93750.h5

Google Drive Link: https://drive.google.com/open?id=1JSUj7mxOwbD-nqseNIVZwWDEDMY_JuO2


#### Final 3D Convolutional Network  

<font color='blue'> Batch_size = 32 and num_epochs = 20 ,
nb_filters = [8,16,32,64] ,
nb_dense = [256, 128, 5]

number of frames =30 ,
image width = 120 ,
image height = 120 
n_classes = 5, 
n_channel = 3 </font><br>
<font color='red'>In 20th epoch-> train accuracy = 79.2% And Validation Accuracy = 81.2%
</font><br>

__.h5 file__<br>
In 20th epoch ==> model_init_2019-09-2216_44_36.385184/model-00020-0.55848-0.79272-0.95341-0.81250.h5

Google Drive Link: https://drive.google.com/open?id=1T27pUpS-hVxGqhUQDwhmzRqRvCW3dXFs
